In [1]:
!pip install tfrecord wandb

In [2]:
# Change this line if you're using Colab to something like '/content/drive/MyDrive/TeamX/'
# where TeamX is just the clone of repository on your Google Drive
# and you have mounted the drive at /content/drive
# See the Tutorial Slides for more detail.

# Works on your local machine but not on Colab!
PROJECT_ROOT = '/notebooks'

# Fix this path and use this one on Colab
# PROJECT_ROOT = '/content/drive/MyDrive/TeamX'

import os
import sys
import torch
import torch.nn as nn

from os.path import join as ospj
sys.path.append(ospj(PROJECT_ROOT, 'src'))
%load_ext autoreload
%autoreload 2

In [11]:
import gc

trainer.model.cpu()
del trainer.model
del trainer

gc.collect()
torch.cuda.empty_cache()
del train_loader
del val_loader
gc.collect()
torch.cuda.empty_cache()

Exception in thread SockSrvRdThr:
  0% 0/736 [06:05<?, ?it/s]Traceback (most recent call last):
  File "/usr/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()

  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/service/server_sock.py", line 112, in run
    shandler(sreq)
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/service/server_sock.py", line 173, in server_record_publish
    iface = self._mux.get_stream(stream_id).interface
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/service/streams.py", line 199, in get_stream
    stream = self._streams[stream_id]
KeyError: '23dbxkga'
  0% 0/736 [02:02<?, ?it/s]
  0% 0/736 [01:30<?, ?it/s]


In [3]:
""" Initialize the trainer and model
"""
import matplotlib.pyplot as plt

from trainers.hppw_trainer import HPPWTrainer
from utils.config_parser import ConfigParser
import data.threeDPW as module_data
from utils.io import seed_everything

# For fair comparisons
seed_everything(100)

%aimport -ConfigParser # Due to an issue of pickle and auto_reload
config = ConfigParser.wo_args(config=ospj(PROJECT_ROOT,'cfgs/project/gradient-config.json'))

datamodule = config.init_obj('train_loader', module_data)
train_loader = datamodule.get_loader()

datamodule = config.init_obj('validation_loader', module_data)
val_loader = datamodule.get_loader()

trainer = HPPWTrainer(config=config, train_loader=train_loader, eval_loader=val_loader)
stats = trainer.train()

plt.plot(stats['loss']['train'], label='train')
plt.plot(stats['loss']['val'], label='val')
plt.title('Classification loss history')
plt.xlabel('Epoch')
plt.ylabel('Classification loss')
plt.legend()
plt.show()


wandb: Currently logged in as: asim-98-12-26 (team-17). Use `wandb login --relogin` to force relogin


==> Start Training Epoch 1/20, lr=0.001000 


Train Epoch: 1 Loss: 0.322437 VIM: 89.01842: :   1% 112/11120 [00:18<26:07,  7.02it/s] 

KeyboardInterrupt: 

In [ ]:
from prettytable import PrettyTable
def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

In [7]:
from torchsummary import summary
from tqdm import tqdm
for i, (history, future) in enumerate(tqdm(val_loader)):
    output = trainer.model(history, future)





  0%|          | 0/203 [00:00<?, ?it/s]


OutOfMemoryError: ignored

In [18]:
print(output.shape)

torch.Size([12, 10, 19, 3])


In [ ]:
import torchinfo
torchinfo.summary(trainer.model)

Layer (type:depth-idx)                                                      Param #
HumanPosePredictorModel                                                     --
├─PoseEncoder: 1-1                                                          --
│    └─Sequential: 2-1                                                      --
│    │    └─PoseEncoderBlock: 3-1                                           3,153,152
│    │    └─PoseEncoderBlock: 3-2                                           3,153,152
├─VisionTransformer: 1-2                                                    768
│    └─Conv2d: 2-2                                                          590,592
│    └─Encoder: 2-3                                                         151,296
│    │    └─MultiInputSequential: 3-3                                       85,054,464
│    │    └─LayerNorm: 3-4                                                  1,536
│    └─Sequential: 2-4                                                      --
│    │    └

In [ ]:
""" Perform test
"""

checkpoint_dir = '0604_104809'
path = ospj(PROJECT_ROOT, f'saved/models/hppw/{checkpoint_dir}/best_val_model.pth')

trainer.load_model(path=path)

result = trainer.evaluate(loader=test_loader)

print(result)


In [ ]:
from models.temporal.encoder import TemporalEncoder, LocalTemporalEncoderBlock

model = TemporalEncoder(
    num_layers=3,
    num_heads=8,
    hidden_dim=256,
    mlp_dim=512
)

In [ ]:
local_feat = torch.randn(size=(32, 15, 197, 256))
global_feat = torch.randn(size=(32, 15, 256))

local_out, global_out = model(local_feat, global_feat)

In [ ]:
local_out

tensor([[[ 0.7680,  0.1498,  2.5715,  ..., -1.7233, -7.7865,  8.6335],
         [ 0.7680,  0.1498,  2.5715,  ..., -1.7233, -7.7865,  8.6336],
         [ 0.7680,  0.1498,  2.5715,  ..., -1.7233, -7.7865,  8.6336],
         ...,
         [ 0.7680,  0.1498,  2.5715,  ..., -1.7233, -7.7865,  8.6335],
         [ 0.7680,  0.1498,  2.5715,  ..., -1.7233, -7.7865,  8.6335],
         [ 0.7680,  0.1498,  2.5715,  ..., -1.7233, -7.7865,  8.6336]],

        [[-1.2756, -7.0257, -1.1117,  ...,  4.7989, -0.6455,  3.4401],
         [-1.2756, -7.0257, -1.1117,  ...,  4.7989, -0.6455,  3.4401],
         [-1.2756, -7.0257, -1.1117,  ...,  4.7989, -0.6455,  3.4401],
         ...,
         [-1.2756, -7.0257, -1.1117,  ...,  4.7989, -0.6455,  3.4401],
         [-1.2756, -7.0257, -1.1117,  ...,  4.7989, -0.6455,  3.4401],
         [-1.2756, -7.0257, -1.1117,  ...,  4.7989, -0.6455,  3.4401]],

        [[-1.1115, -3.7773, -9.5381,  ..., -0.7923,  2.3916, -3.7397],
         [-1.1115, -3.7773, -9.5381,  ..., -0

In [ ]:
from torchsummary import summary

summary(model, input_data=[local_feat, global_feat])

Layer (type:depth-idx)                             Output Shape              Param #
├─MultiInputSequential: 1-1                        [-1, 197, 256]            --
|    └─TemporalEncoderBlock: 2-1                   [-1, 14, 197, 256]        --
|    |    └─LocalTemporalEncoderBlock: 3-1         [-1, 14, 197, 256]        528,128
|    |    └─GlobalTemporalEncoderBlock: 3-2        [-1, 15, 256]             527,104
|    └─TemporalEncoderBlock: 2-2                   [-1, 13, 197, 256]        --
|    |    └─LocalTemporalEncoderBlock: 3-3         [-1, 13, 197, 256]        528,128
|    |    └─GlobalTemporalEncoderBlock: 3-4        [-1, 15, 256]             527,104
|    └─TemporalEncoderBlock: 2-3                   [-1, 197, 256]            --
|    |    └─LocalTemporalEncoderBlock: 3-5         [-1, 197, 256]            528,128
|    |    └─GlobalTemporalEncoderBlock: 3-6        [-1, 15, 256]             527,104
Total params: 3,165,696
Trainable params: 3,165,696
Non-trainable params: 0
Total mul

Layer (type:depth-idx)                             Output Shape              Param #
├─MultiInputSequential: 1-1                        [-1, 197, 256]            --
|    └─TemporalEncoderBlock: 2-1                   [-1, 14, 197, 256]        --
|    |    └─LocalTemporalEncoderBlock: 3-1         [-1, 14, 197, 256]        528,128
|    |    └─GlobalTemporalEncoderBlock: 3-2        [-1, 15, 256]             527,104
|    └─TemporalEncoderBlock: 2-2                   [-1, 13, 197, 256]        --
|    |    └─LocalTemporalEncoderBlock: 3-3         [-1, 13, 197, 256]        528,128
|    |    └─GlobalTemporalEncoderBlock: 3-4        [-1, 15, 256]             527,104
|    └─TemporalEncoderBlock: 2-3                   [-1, 197, 256]            --
|    |    └─LocalTemporalEncoderBlock: 3-5         [-1, 197, 256]            528,128
|    |    └─GlobalTemporalEncoderBlock: 3-6        [-1, 15, 256]             527,104
Total params: 3,165,696
Trainable params: 3,165,696
Non-trainable params: 0
Total mul

In [ ]:

""" Initialize the trainer and model
"""
import matplotlib.pyplot as plt

from trainers.hppw_trainer import HPPWTrainer

trainer = HPPWTrainer(config=config, train_loader=train_loader, eval_loader=val_loader)
# stats = trainer.train()

plt.plot(stats['loss']['train'], label='train')
plt.plot(stats['loss']['val'], label='val')
plt.title('Classification loss history')
plt.xlabel('Epoch')
plt.ylabel('Classification loss')
plt.legend()
plt.show()


KeyError: 'wandb'